In [1]:
import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

from matplotlib import gridspec

from sklearn.preprocessing import scale

from tqdm.notebook import tqdm

import json

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader

from torch.utils.data import Dataset

from sklearn.utils import shuffle
from scipy.special import softmax

from PIL import Image

from PIL import ImageDraw

from skimage import morphology

import pickle

from sklearn.model_selection import ParameterGrid

In [3]:
from mixturelib.mixture import MixtureEM
from mixturelib.local_models import EachModelLinear, EachModel
from mixturelib.hyper_models import HyperExpertNN, HyperModelDirichlet

from mixturelib.regularizers import RegularizeModel

In [4]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [6]:
def get_data(beta=0.1, l=200):
    np.random.seed(0)

    t = np.linspace(-2, 2, l)

    p = 0.5
    alpha = 0.9
    x0 = 0
    y0 = 0

    x=x1=((t**2)/(2*p))*np.cos(alpha)-t*np.sin(alpha)+x0
    y=y1=((t**2)/(2*p))*np.sin(alpha)+t*np.cos(alpha)+y0
    X = np.vstack([x*y, y**2, x, y, np.ones_like(x)]).T
    Y = x**2
    w1 = np.linalg.inv(X.T@X)@X.T@Y
    x1 += beta*np.random.randn(l)
    y1 += beta*np.random.randn(l)


    p = 0.3
    alpha = 2
    x0 = 5
    y0 = 5
    x=x2=((t**2)/(2*p))*np.cos(alpha)-t*np.sin(alpha)+x0
    y=y2=((t**2)/(2*p))*np.sin(alpha)+t*np.cos(alpha)+y0
    X = np.vstack([x*y, y**2, x, y, np.ones_like(x)]).T
    Y = x**2
    w2 = np.linalg.inv(X.T@X)@X.T@Y
    x2 += beta*np.random.randn(l)
    y2 += beta*np.random.randn(l)


    x = np.hstack([x1, x2])
    y = np.hstack([y1, y2])
    
    return x, y, w1, w2

In [7]:
grid = ParameterGrid({'beta': np.linspace(0.001, 0.5, 5), 
                      'gamma': np.linspace(0.0001, 1.0, 5)})

In [8]:
K = 2
n = 5
epochs = 5

experiment_dict = dict()

experiment_dict['betas'] = grid.param_grid[0]['beta']
experiment_dict['gammas'] = grid.param_grid[0]['gamma']

experiment_dict['K'] = K
experiment_dict['n'] = n
experiment_dict['epochs'] = epochs

experiment_dict['results'] = dict()

for item in tqdm(grid):
    key = json.dumps(item)
    experiment_dict['results'][key] = dict()

    x, y, w1, w2 = get_data(beta=item['beta'])
    
    experiment_dict['results'][key]['x'] = x
    experiment_dict['results'][key]['y'] = y
    experiment_dict['results'][key]['w_real'] = [w1, w2]
    
    X = np.vstack([x*y, y**2, x, y, np.ones_like(x)]).T
    Y = x**2

    X_tr = torch.FloatTensor(X).to(device)
    Y_tr = torch.FloatTensor(Y).view([-1,1]).to(device)


    LIST_A = [torch.FloatTensor([item['gamma']]*n).to(device) for _ in range(K)]
    LIST_w = [torch.FloatTensor(w1).view(-1,1).to(device),  
              torch.FloatTensor(w2).view(-1,1).to(device)]


    LoM = []
    for k in range(K):
        LoM.append(EachModelLinear(input_dim = n, 
                                   device = device, 
                                   A = LIST_A[k], 
                                   w = LIST_w[k],
                                   OptimizedHyper={}))


    HypPar = {'beta': item['beta'], 'alpha': 1e-5}


    HpMd = HyperExpertNN(input_dim=5, output_dim = K, device = device)

    model = MixtureEM(HyperParameters = HypPar,
                      HyperModel = HpMd, 
                      ListOfModels = LoM,
                      device = device, model_type='sample')

    experiment_dict['results'][key]['models'] = []
    for _ in range(epochs):
        model.fit(X_tr, Y_tr, 1)
        experiment_dict['results'][key]['models'].append(model)

    w1_pred = LoM[0].W.view(-1).cpu().numpy()
    w2_pred = LoM[1].W.view(-1).cpu().numpy()

    experiment_dict['results'][key]['w_pred'] = [w1_pred, w2_pred]
    experiment_dict['results'][key]['model'] = model

In [9]:
with open('./dump/experiment.pkl', 'wb') as f:
    pickle.dump(experiment_dict, f)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
